In [1]:
import requests
import pandas as pd
url = "https://www.starbucks.co.kr/store/getStore.do?r=86N465IIB2"
payload = {"in_biz_cds": "0",
"in_scodes": "0",
"ins_lat": "37.4745",
"ins_lng": "126.8918",
"search_text": "",
"p_sido_cd": "01",
"p_gugun_cd": "",
"in_distance": "0",
"in_biz_cd": "",
"isError": "true",
"searchType": "C",
"set_date": "",
"all_store": "0",
"T03": "0",
"T01": "0",
"T27": "0",
"T12": "0",
"T09": "0",
"T30": "0",
"T05": "0",
"T22": "0",
"T21": "0",
"T36": "0",
"T43": "0",
"Z9999": "0",
"T64": "0",
"T66": "0",
"P02": "0",
"P10": "0",
"P50": "0",
"P20": "0",
"P60": "0",
"P30": "0",
"P70": "0",
"P40": "0",
"P80": "0",
"whcroad_yn": "0",
"P90": "0",
"P01": "0",
"new_bool": "0",
"iend": "1000",
"rndCod": "XW4W0LA4X8",}
r = requests.post(url, data=payload)
star = r.json()
star_df = pd.DataFrame(star['list'])
star_df

,seq,sido_cd,sido_nm,gugun_cd,gugun_nm,code_order,view_yn,store_num,sido,gugun,...,t30,t36,t27,t29,t43,t48,z9999,t64,t66,p02
0,0,None,None,None,None,None,None,None,None,None,...,0,0,0,0,0,0,0,0,0,0
1,0,None,None,None,None,None,None,None,None,None,...,0,0,0,0,0,0,0,0,0,0
2,0,None,None,None,None,None,None,None,None,None,...,0,0,0,0,0,0,0,0,0,0
3,0,None,None,None,None,None,None,None,None,None,...,0,0,0,0,0,0,0,0,0,0
4,0,None,None,None,None,None,None,None,None,None,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
638,0,None,None,None,None,None,None,None,None,None,...,0,0,0,0,0,0,0,0,0,0
639,0,None,None,None,None,None,None,None,None,None,...,0,0,0,0,0,0,0,0,0,0
640,0,None,None,None,None,None,None,None,None,None,...,0,0,0,0,0,0,0,0,0,0
641,0,None,None,None,None,None,None,None,None,None,...,0,0,0,0,0,0,0,0,0,0


In [13]:
#사용할 데이터만 가져옴
star_df2 = star_df[["s_name" , 's_biz_code',  "addr",  "defaultimage", "doro_address", "lat",  "lot", "open_dt", ]].copy()

In [14]:
#open_dt를 날짜타입으로 변경
from datetime import datetime, date, time
star_df2['open_dt_datetype'] = star_df2['open_dt'].apply(lambda x : date(int(x[0:4]), int(x[4:6]), int(x[6:])))
star_df2.open_dt_datetype2 = star_df2.open_dt.apply((lambda x : datetime.strptime(x,'%Y%m%d')))

/tmp/ipykernel_23098/3782849214.py:4: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  star_df2.open_dt_datetype2 = star_df2.open_dt.apply((lambda x : datetime.strptime(x,'%Y%m%d')))


In [15]:
#open_year 추가
star_df2['open_year'] = star_df2['open_dt_datetype'].apply(lambda x : x.year)

In [16]:
open_inyear = star_df2['open_year'].value_counts()
open_inyear

open_year
2018    48
2014    48
2017    47
2016    47
2021    45
2022    41
2019    37
2015    37
2024    35
2013    34
2020    34
2012    30
2023    24
2011    20
2010    16
2025    15
2008    13
2005    12
2009    11
2007    10
2004     9
2006     8
2002     8
2003     7
2001     5
1999     1
2000     1
Name: count, dtype: int64

In [8]:
time_url ="https://www.starbucks.co.kr/store/getStoreTime.do"

In [10]:
time_payload = {
    "in_biz_cd" : "4247",
    "in_store_type" : "C",
    "rndCod" : "0ZKPG4SMG6",
    }

In [18]:
view_url = "https://www.starbucks.co.kr/store/getStoreView.do"
view_payload = {'in_biz_cd' : '3672',
                'rndCod' : '0S54HHYI84'}

In [19]:
import time , tqdm
total =[]
for _, data in star_df2.iterrows():
    biz_code = data['s_biz_code']
    view_payload['in_biz_cd'] = biz_code
    total.append(pd.DataFrame(requests.post(view_url, data=view_payload).json()['view']))
    if _ % 100 == 99:
        time.sleep(3)

In [20]:
t1 = pd.concat(total)

In [21]:
star_parking_df2 = pd.read_pickle("starbucks_park.pkl")
star_df3 = pd.merge(star_df2, star_parking_df2, left_on = ['s_biz_code'], right_on=['s_biz_code'], how='inner')

In [22]:
#주차불가 갯수
sum(star_df3.park_info.str.find("불가") > -1)

381

In [28]:
#결측치 있음
print(star_df3.shape[0])
print(sum(star_df3['park_info'].notnull()))

643
642


In [25]:
star_df3.shape

(643, 12)

In [29]:
#map_desc에 주차 가능한지써져있어서 결측치에 대입입
star_df3[star_df3.park_info.isnull()]['map_desc'].values[0]
star_df3.loc[ star_df3.park_info.isnull(), 'park_info'] = star_df3[star_df3.park_info.isnull()]['map_desc'].values[0]

In [30]:
star_df3.park_info.isnull().sum()

np.int64(0)

In [ ]:
star_df3.loc[star_df3.park_info.str.find("불가") == -1, 'park_yn'] = True
#park_yn 칼럼을 만들어서 park_info에 불가가 없으면 True 대입
#park_yn의 빈 값들에 False를 채우면됨

star_df3.park_yn.fillna(False)
#fillna 는 결측치를 채워서 반환

star_df3['park_yn'] = star_df3.park_yn.fillna(False)
#이렇게 다시 대입해줘야함

/tmp/ipykernel_23098/1497391048.py:1: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  star_df3['park_yn'] = star_df3.park_yn.fillna(False)
